In [1]:
import transformers

/Users/nfanlo/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#No funciona import trainer, trainingArguments ni earlystoppingcallback
from transformers import EarlyStoppingCallback

2023-03-01 02:24:13.249174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: Failed to import transformers.trainer_callback because of the following error (look up to see its traceback):
WRONG PACKAGE. Please install the package from Neuron Repository - pip.repos.neuron.amazonaws.com